In [1]:
from graphframes import *
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import col
import os
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GraphAnalysis") \
    .getOrCreate()

In [2]:
v = spark.read.csv("data/2_people_data_2k.csv", header=True, inferSchema=True)
e = spark.read.csv("data/2_connections_data_300k.csv", header=True, inferSchema=True)

In [3]:
g = GraphFrame(v, e)
g.persist(StorageLevel.MEMORY_AND_DISK)

/usr/local/spark/python/pyspark/sql/dataframe.py:169: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


GraphFrame(v:[id: int, name: string ... 1 more field], e:[src: int, dst: int ... 1 more field])

a) Ai là người có nhiều bạn nhất (tổng bậc cao nhất)?

In [5]:
from pyspark.sql.functions import col

# Tổng bậc = số lần xuất hiện trong src hoặc dst
degrees = g.inDegrees.union(g.outDegrees).groupBy("id").sum("inDegree") \
    .withColumnRenamed("sum(inDegree)", "total_degree") \
    .join(g.vertices, on="id")

degrees.orderBy(col("total_degree").desc()).show(1)

+----+------------+------------+---+
|  id|total_degree|        name|age|
+----+------------+------------+---+
|1768|         353|Mai Bảo Hạnh| 61|
+----+------------+------------+---+
only showing top 1 row



b) Ai có tầm ảnh hưởng lớn nhất theo PageRank?

In [6]:
pagerank = g.pageRank(resetProbability=0.15, maxIter=10)
pagerank.vertices.select("id", "name", "pagerank").orderBy(col("pagerank").desc()).show(1)

+---+---------+------------------+
| id|     name|          pagerank|
+---+---------+------------------+
|918|Vũ Phương|1.2482510238869442|
+---+---------+------------------+
only showing top 1 row



c) Liệt kê tất cả các "bạn của bạn" của một người cụ thể

In [7]:
target_name = "Nguyễn An"
target_id = g.vertices.filter(col("name") == target_name).select("id").first()["id"]

fof = g.find("(a)-[]->(b); (b)-[]->(c)") \
    .filter(f"a.id = '{target_id}' AND c.id != a.id") \
    .selectExpr("a.id as user", "b.id as friend", "c.id as friend_of_friend") \
    .join(g.vertices.alias("c_v"), col("friend_of_friend") == col("c_v.id")) \
    .select("c_v.name").distinct()

print(f"Các 'bạn của bạn' của {target_name}:")
fof.show(truncate=False)


Các 'bạn của bạn' của Nguyễn An:
+----------------+
|name            |
+----------------+
|Phạm Phú Huy    |
|Dương Bảo       |
|Trần An         |
|Phạm Xuân Phương|
|Mai Trí Tùng    |
|Đặng Đức Quang  |
|Phạm Hữu Hạnh   |
|Vũ Hữu Hà       |
|Trần Quang      |
|Hoàng Hải Trọng |
|Mai Văn An      |
|Dương Xuân Lâm  |
|Vũ Ngọc         |
|Trần Xuân Nhật  |
|Vũ Đức Trọng    |
|Bùi Dương       |
|Phạm Ánh        |
|Đặng Mai        |
|Nguyễn Linh     |
|Bùi Châu        |
+----------------+
only showing top 20 rows



d) Đồ thị có bao nhiêu thành phần liên thông? Nhóm nào lớn nhất?

In [10]:
spark.sparkContext.setCheckpointDir("/tmp/graphframes-checkpoint")

cc = g.connectedComponents()
group_counts = cc.groupBy("component").count().orderBy(col("count").desc())

# Tổng số thành phần
print("Số thành phần liên thông:", group_counts.count())

# Thành phần lớn nhất
print("Thành phần liên thông lớn nhất:")
group_counts.show(1)

Số thành phần liên thông: 1
Thành phần liên thông lớn nhất:
+---------+-----+
|component|count|
+---------+-----+
|        1| 2000|
+---------+-----+



e) Tìm đường đi ngắn nhất từ 'Vũ Linh' đến 'Phạm Phúc'

In [ ]:
from_name = "Vũ Linh"
to_name = "Phạm Phúc"

from_id = g.vertices.filter(col("name") == from_name).select("id").first()["id"]
to_id = g.vertices.filter(col("name") == to_name).select("id").first()["id"]

paths = g.bfs(fromExpr=f"id = '{from_id}'", toExpr=f"id = '{to_id}'", maxPathLength=5)
paths.show(truncate=False)

+-----------------+------------------+---------------------------+------------------+-------------------+
|from             |e0                |v1                         |e1                |to                 |
+-----------------+------------------+---------------------------+------------------+-------------------+
|{20, Vũ Linh, 65}|{20, 588, follow} |{588, Bùi Tùng, 25}        |{588, 11, follow} |{11, Phạm Phúc, 68}|
|{20, Vũ Linh, 65}|{20, 1621, follow}|{1621, Phạm Tùng, 29}      |{1621, 11, friend}|{11, Phạm Phúc, 68}|
|{20, Vũ Linh, 65}|{20, 402, friend} |{402, Trần Hoàng Hà, 40}   |{402, 11, friend} |{11, Phạm Phúc, 68}|
|{20, Vũ Linh, 65}|{20, 764, follow} |{764, Trần Quang Trọng, 67}|{764, 11, follow} |{11, Phạm Phúc, 68}|
|{20, Vũ Linh, 65}|{20, 347, follow} |{347, Phạm Dương, 58}      |{347, 11, friend} |{11, Phạm Phúc, 68}|
|{20, Vũ Linh, 65}|{20, 1240, follow}|{1240, Hoàng Dũng, 40}     |{1240, 11, follow}|{11, Phạm Phúc, 68}|
|{20, Vũ Linh, 65}|{20, 568, friend} |{568, Tr